In [1]:
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
plt.rcParams['image.cmap'] = 'gray'

In [5]:
import os

import numpy as np
import sklearn.metrics as metrics
from keras import backend as K
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.datasets import cifar10
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

import modelresnext as modelresnext
print(dir(modelresnext))


batch_size = 128
nb_classes = 10
nb_epoch = 100

img_rows, img_cols = 32, 32
img_channels = 3

img_dim = (img_channels, img_rows, img_cols) if K.image_dim_ordering() == "th" else (img_rows, img_cols, img_channels)
depth = 29
cardinality = 8
width = 16

model = "mine" # "other"
if model== "mine":
    weights_file = "weights/mine.h5"
    model= modelresnext.residual_network(img_dim,nb_classes,cardinality,width)
else:
    weights_file = "weights/other.h5"
    model = modelresnext.residual_network(img_dim, depth=depth, cardinality=cardinality, width=width, weights=None, classes=nb_classes)

print("Model created")

model.summary()

optimizer = Adam(lr=1e-3)  # Using Adam instead of SGD to speed up training
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])
print("Finished compiling")
print("Building model...")

(trainX, trainY), (testX, testY) = cifar10.load_data()

trainX = trainX.astype('float32')
testX = testX.astype('float32')

trainX /= 255.
testX /= 255.

Y_train = np_utils.to_categorical(trainY, nb_classes)
Y_test = np_utils.to_categorical(testY, nb_classes)

generator = ImageDataGenerator(rotation_range=15,
                               width_shift_range=5./32,
                               height_shift_range=5./32,
                               horizontal_flip=True)

generator.fit(trainX, seed=0)

out_dir = "weights/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Load model


if False and os.path.exists(weights_file):
    model.load_weights(weights_file)
    print("Model loaded.")

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1),
                               cooldown=0, patience=10, min_lr=1e-6)

model_checkpoint = ModelCheckpoint(weights_file, monitor="val_acc", save_best_only=True,
                                   save_weights_only=True, mode='auto')

callbacks = [lr_reducer, model_checkpoint]

print("Done")

['Activation', 'BatchNormalization', 'CIFAR_TF_WEIGHTS_PATH', 'CIFAR_TF_WEIGHTS_PATH_NO_TOP', 'CIFAR_TH_WEIGHTS_PATH', 'CIFAR_TH_WEIGHTS_PATH_NO_TOP', 'Conv2D', 'Dense', 'GlobalAveragePooling2D', 'GlobalMaxPooling2D', 'IMAGENET_TF_WEIGHTS_PATH', 'IMAGENET_TF_WEIGHTS_PATH_NO_TOP', 'IMAGENET_TH_WEIGHTS_PATH', 'IMAGENET_TH_WEIGHTS_PATH_NO_TOP', 'Input', 'K', 'Lambda', 'MaxPooling2D', 'Model', 'ResNextImageNet', '__bottleneck_block', '__builtins__', '__cached__', '__create_res_next', '__create_res_next_imagenet', '__file__', '__grouped_convolution_block', '__initial_conv_block', '__initial_conv_block_imagenet', '__loader__', '__name__', '__package__', '__spec__', '_obtain_input_shape', 'add', 'concatenate', 'convert_all_kernels_in_model', 'get_file', 'get_source_inputs', 'l2', 'residual_network', 'warnings']


[autoreload of modelresnext failed: Traceback (most recent call last):
  File "/data/dev/experiments/.env/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/data/dev/experiments/.env/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/usr/lib/python3.5/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.5/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 626, in _exec
  File "<frozen importlib._bootstrap_external>", line 661, in exec_module
  File "<frozen importlib._bootstrap_external>", line 767, in get_code
  File "<frozen importlib._bootstrap_external>", line 727, in source_to_code
  File "<frozen importlib._bootstrap>", line 222, in _call_with_frames_removed
  File "/data/dev/experiments/resnext_implementation/model

ValueError: Depth of the network must be such that (depth - 2)should be divisible by 9.

In [ ]:
print("Training...")
model.fit_generator(generator.flow(trainX, Y_train, batch_size=batch_size),
                    steps_per_epoch=len(trainX) // batch_size,
                    epochs=nb_epoch,
                    callbacks=callbacks,
                    validation_data=(testX, Y_test),
                    validation_steps=testX.shape[0] // batch_size, verbose=1)

Training...
Epoch 1/100
390/390 [==============================] - 237s 609ms/step - loss: 1.9653 - acc: 0.2718 - val_loss: 1.6992 - val_acc: 0.3815
Epoch 2/100
390/390 [==============================] - 235s 603ms/step - loss: 1.6807 - acc: 0.3789 - val_loss: 1.4729 - val_acc: 0.4636
Epoch 3/100
390/390 [==============================] - 235s 604ms/step - loss: 1.5523 - acc: 0.4317 - val_loss: 1.4179 - val_acc: 0.4762
Epoch 4/100
390/390 [==============================] - 236s 604ms/step - loss: 1.4801 - acc: 0.4585 - val_loss: 1.3368 - val_acc: 0.5072
Epoch 5/100
390/390 [==============================] - 235s 604ms/step - loss: 1.4296 - acc: 0.4789 - val_loss: 1.2762 - val_acc: 0.5346
Epoch 6/100
390/390 [==============================] - 236s 604ms/step - loss: 1.3796 - acc: 0.4973 - val_loss: 1.2366 - val_acc: 0.5501
Epoch 7/100
390/390 [==============================] - 235s 603ms/step - loss: 1.3475 - acc: 0.5122 - val_loss: 1.2271 - val_acc: 0.5553
Epoch 8/100
390/390 [========

In [ ]:

yPreds = model.predict(testX)
yPred = np.argmax(yPreds, axis=1)
yTrue = testY

accuracy = metrics.accuracy_score(yTrue, yPred) * 100
error = 100 - accuracy
print("Accuracy : ", accuracy)
print("Error : ", error)